### Targeted information retrieval

We have seen how to parse a webpage, which retrieves the information without distinction.

But, in general, the purpose of scraping is to automate the collection of targeted information on the web.


In [ ]:
from bs4 import BeautifulSoup
import requests

Let's say I want to scrape the description of the latest movies released in theaters.

So we'll go to the Allociné website and try to find the tags that will give me links to the specific pages of these movies to get their summaries.

#### Recovery of the url of the pages of films newly released in the theaters

In [ ]:
url = "http://www.allocine.fr/"
r = requests.get(url)
print(url, r.status_code)
soup = BeautifulSoup(r.content, "html")
soup

On your web browser (Chrome, Firefox,...), you can use the "inspect" function (right click -> inspect) and drag your mouse to the areas of the page that interest you. At the same time you will see the part of the HTML script highlighted that corresponds to the area that interests you.

That's how you find the tags that you are interested in.

I notice that the relative link of the web page specific to the new movie on the poster is stored in these tags:

```html
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=235582.html" title="Le Grand Bain">Le Grand Bain</a>
```


In [ ]:
for p in soup.find_all("a"):
    print(p.text)

This time, the site is more difficult to "extract". Let's use much more specific parameters to the search function `find_all`.

In [ ]:
# In addition to the tag `a`, which is easily identifiable, we notice some additional
# information such as the value of the class variable of these identical tags.
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    print(elem)

#### Recovery of `href`

We have noticed the presence of `href` links to the pages that interest us. Let's go get them back:

In [ ]:
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    print(elem.get("href"))

Can you retrieve the titles for me by searching for `"title"` in the items of the previous list?

#### Get summary

Let's start by building the URL that we will use to retrieve the summaries.

Start by putting the `href` values in a list of links.


In [ ]:
links = []
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    links.append(elem.get("href"))
links

The absolute URL of the movie pages we are looking for are built in this form: http://www.allocine.fr/film/fichefilm_gen_cfilm=243835.html

It is therefore necessary to take the previous list and build the absolute URLs for our search
It's up to you to play.

NB: Do not take the links for the shows(series). We only want movies.

In [ ]:
movie_links = ["http://www.allocine.fr" + elem for elem in links if "film" in elem]
movie_links

Finally, on each page, the title and synopsis must be retrieved. Let's try for a movie, the first of the list.

In [ ]:
url = movie_links[0]
r = requests.get(url)
print(url, r.status_code)
soup = BeautifulSoup(r.content, "html")
soup

For title 
```html
<div class="titlebar-title titlebar-title-lg">Le Grand Bain</div>
```
For the synopsis

```html
<section class="section ovw ovw-synopsis" id="synopsis-details">

      <div class="content-txt">
 
              C’est dans les couloirs de leur piscine municipale que Bertrand, Marcus, Simon, Laurent, Thierry et les autres s’entraînent sous l’autorité toute relative de Delphine, ancienne gloire des bassins. Ensemble, ils se sentent libres et utiles. Ils vont mettre toute leur énergie dans une discipline jusque-là propriété de la gent féminine : la natation synchronisée. Alors, oui c’est une idée plutôt bizarre, mais ce défi leur permettra de trouver un sens à leur vie...
      </div>
    
      </section>
```

In [ ]:
for elem in soup.find_all("div", attrs={"class": "titlebar-title titlebar-title-lg"}):
    # Just like that
    print(elem.text)

# Get the synopsis section
for elem in soup.find_all("section", attrs={"id": "synopsis-details"}):
    # Get the text of the synopsis
    for elem2 in elem.find_all("div", attrs={"class":"content-txt"}):
        # Just like that
        print(elem2.text)

1) Automate this script for the entire list of movies.

2) Put the information in three lists (`film_links`, `title`, `synopsis`).

3) Create a `DataFrame` object from the `pandas` library that you will have to install. This dataframe will have to include these three informations in three columns.

4) Save this dataframe in a CSV file.

And here's your first real scrap, you're real hackers now.

In [ ]:
import time
import random
from random import randint

title = []
synopsis = []

# Here are the things you will have to do for all links:
# - Slow down the frequency of requests to avoid being identified and therefore banned from the website.
# Use `time.sleep(random.uniform(1.0, 2.0))`
# - Get request object from URL
# - Extract the content into a variable using BeautifulSoup
# - Get title
# - Get synopsis

# Check the length of the lists before creating the dataframe
len(title), len(synopsis), len(movie_links)

In [ ]:
import pandas as pd

df = pd.DataFrame({"Title": title})
df["Synopsis"] = synopsis
df["Links"] = movie_links

In [ ]:
df.to_csv("./assets/allo_cine.csv", index=False)